In [2]:
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=UserWarning)
import train_flow
import importlib
importlib.reload(train_flow)
import pandas as pd
import numpy as np
from xgboost import XGBClassifier

# Feature Engineering for net new features

In [4]:
ticker = 'QQQ'
min_feats = "N"
returns = [1, 2, 3, 5, 10, 20, 30]

df_daily, feature_sets, return_cols, daily_cols, sets, set_names, feature_cols = train_flow.import_data(ticker, min_feats, returns) 

# Add any new features
df_daily[[f"{c}_sum10" for c in df_daily.columns if c.startswith("Past_Return_")]] = (df_daily.sort_values(by="Date", ascending=True).filter(like="Past_Return_").rolling(10, min_periods=1).sum())
past_ret_cols = [c for c in df_daily.columns if c.startswith("Past_Return%") or c.endswith("sum10")]
past_perc_cols = [c for c in df_daily.columns if c.startswith("Past_Return%")]
past_sum_cols = [c for c in df_daily.columns if c.endswith("sum10")]
df_main = df_daily[df_daily['Date'] <= '2026-02-06'].copy() #cutoff date for baseline performance metrics

Available Feature Sets: dict_keys(['ma', 'rsi', 'macd', 'volume', 'atr_adx', 'volatility', 'vix_skew', 'experimental_slope'])


# Run Train_Flow

In [ ]:
# -----------------------------
# Handling of new features and permutation importance
# -----------------------------
pi_handlings = ['include_new'] #'exclude_new', 'include_new', 'run_separately'
train_flow.pi_handling_test(pi_handlings)
new_features = [] #past_sum_cols

# -----------------------------
# Run grid (feature sets x horizon x train_years, etc.)
# -----------------------------
target_horizons = [2, 10, 30]#[2, 5, 10, 20, 30]
train_years = [5]
pi_years = [1.5]
min_feats = [8]
days_assessed = 5
test_day = 1
feature_cols = feature_cols
models = {"xgboost-3": XGBClassifier(n_estimators=300, random_state=42, n_jobs=-1)}
list_name = 'kitchen_sink'

results_df = train_flow.run_train_flow(test_day, days_assessed, target_horizons, pi_handlings, feature_cols, df_main, models,
                   train_years, pi_years, min_feats, list_name, new_features)
results_df

Running for horizon 2 | include_new
Running for horizon 10 | include_new
Running for horizon 30 | include_new


,feature_set,horizon


# Run Performance_Flow

In [ ]:
import performance_flow
importlib.reload(performance_flow)

results_file_name = "test.csv"
return_cols, perf_df = performance_flow.import_data(results_file_name, df_daily)

min_th = 0.55
cov_th = 0.75
composite_score = performance_flow.run_performance(perf_df, min_th, cov_th)
bucket_df = performance_flow.bucket_scores(df_daily, perf_df, returns, min_th)

,horizon,model,train_years,feature_set,pi_size,min_feats,acc_1_c,acc_-1_c,n_1_c,n_-1_c,...,acc_3+_o,acc_-3+_o,n_3+_o,n_-3+_o,bal_acc_1_o,bal_acc_2_o,bal_acc_3_o,bal_acc_3p_o,wba_close,wba_open
6,2,xgboost-3,4,past_ret_cols-baseline,1.5,8,0.750000,0.176471,16.0,17.0,...,0.538462,0.571429,26.0,14.0,0.639706,0.593407,0.540404,0.554945,0.57,0.59
1,2,xgboost-3,4,experimental_slope-baseline,1.5,8,0.750000,0.470588,16.0,17.0,...,0.538462,0.500000,26.0,14.0,0.579044,0.478022,0.530303,0.519231,0.52,0.53
7,2,xgboost-3,4,rsi_macd-baseline,1.5,8,0.687500,0.411765,16.0,17.0,...,0.653846,0.571429,26.0,14.0,0.518382,0.436813,0.474747,0.612637,0.50,0.50
9,2,xgboost-3,4,volu-baseline,1.5,8,0.625000,0.352941,16.0,17.0,...,0.615385,0.642857,26.0,14.0,0.459559,0.403846,0.641414,0.629121,0.49,0.51
2,2,xgboost-3,4,ma_lag-baseline,1.5,8,0.500000,0.411765,16.0,17.0,...,0.692308,0.571429,26.0,14.0,0.547794,0.335165,0.742424,0.631868,0.47,0.55
8,2,xgboost-3,4,vix_skew-baseline,1.5,8,0.562500,0.352941,16.0,17.0,...,0.461538,0.571429,26.0,14.0,0.457721,0.480769,0.484848,0.516484,0.46,0.48
0,2,xgboost-3,4,atr_adxvola-baseline,1.5,8,0.562500,0.352941,16.0,17.0,...,0.384615,0.500000,26.0,14.0,0.579044,0.409341,0.393939,0.442308,0.46,0.47
3,2,xgboost-3,4,ma_num-baseline,1.5,8,0.625000,0.470588,16.0,17.0,...,0.653846,0.642857,26.0,14.0,0.334559,0.445055,0.348485,0.648352,0.44,0.42
4,2,xgboost-3,4,ma_rel-baseline,1.5,8,0.812500,0.529412,16.0,17.0,...,0.653846,0.500000,26.0,14.0,0.419118,0.337912,0.292929,0.576923,0.43,0.40
5,2,xgboost-3,4,ma_sma-baseline,1.5,8,0.625000,0.647059,16.0,17.0,...,0.730769,0.571429,26.0,14.0,0.305147,0.332418,0.484848,0.651099,0.42,0.41
